In [3]:
import pandas as pd
import numpy as np
import re
# import pycountry
from textblob import TextBlob
from calendar import month_name
from collections import Counter


In [1313]:
with open('oil_spill_2000_2017.txt') as f:
    text = f.read()

In [1314]:
splitted_text = text.splitlines()

In [1315]:
text

'== 2000s ==\n\n=== 2000 ===\n* On January 10, approximately 100 barrels of jet fuel were discharged from [[Plantation Pipeline]] in [[Newington, Virginia]], some of which entered into [[Accotink Creek]] and its adjoining shorelines. The failure resulted from a failed gasket on an interface detector.<ref name="epa.gov">{{cite web|url=http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |title=UNITED STATES OF AMERICA, and STATE OF NORTH CAROLINA, Plaintiffs, v. PLANTATION PIPE LINE COMPANY, Civil Action No. 3:08-cv-500 |accessdate=June 14, 2012 |deadurl=yes |archiveurl=https://web.archive.org/web/20131028061446/http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |archivedate=October 28, 2013 |df= }}</ref>\n* On January 21, a [[Chevron Corporation|Chevron]] pipeline leaked from a welding flaw near [[Corinne, Utah]], spilling about 100 barrels of Diesel fuel. The product spread over 38 acres of salt flat and wetlands used by 

In [1316]:
splitted_text[:5]

['== 2000s ==',
 '',
 '=== 2000 ===',
 '* On January 10, approximately 100 barrels of jet fuel were discharged from [[Plantation Pipeline]] in [[Newington, Virginia]], some of which entered into [[Accotink Creek]] and its adjoining shorelines. The failure resulted from a failed gasket on an interface detector.<ref name="epa.gov">{{cite web|url=http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |title=UNITED STATES OF AMERICA, and STATE OF NORTH CAROLINA, Plaintiffs, v. PLANTATION PIPE LINE COMPANY, Civil Action No. 3:08-cv-500 |accessdate=June 14, 2012 |deadurl=yes |archiveurl=https://web.archive.org/web/20131028061446/http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |archivedate=October 28, 2013 |df= }}</ref>',
 '* On January 21, a [[Chevron Corporation|Chevron]] pipeline leaked from a welding flaw near [[Corinne, Utah]], spilling about 100 barrels of Diesel fuel. The product spread over 38 acres of salt flat and wetl

## Extract date

In [1317]:
def segment_timeline(splitted_text):
    index_memory = []
    for i,paragraph in enumerate(splitted_text):
        target_text = re.findall(r"=== \d{4} ===", paragraph)
        if target_text != []:
    #         print(target_text)
            index_memory.append(i)
    return index_memory

index_memory = segment_timeline(splitted_text)

In [1318]:
def add_year_to_each_paragraph(index_memory, start_year=2000):
    y_text = []
    lst = []
    year = start_year
    for i in range(len(index_memory)- 1): # go over each year
        lst = []
        lst.append(splitted_text[index_memory[i]:index_memory[i+1]]) # get number of accidents in that specific year
        lst = [item for items in lst for item in items]
        for j,v in enumerate(lst):
            target_text = re.findall(r'^\* [Oo]n', str(v))
            if target_text != []:
                valid_text = re.sub(r'\* ',"* " + str(year)+" ", v) 
                y_text.append(valid_text) # append valid paragraph with corresponding year
        year+=1
    return y_text

In [1319]:
splitted_text = add_year_to_each_paragraph(index_memory)

In [1320]:
splitted_text

['* 2000 On January 10, approximately 100 barrels of jet fuel were discharged from [[Plantation Pipeline]] in [[Newington, Virginia]], some of which entered into [[Accotink Creek]] and its adjoining shorelines. The failure resulted from a failed gasket on an interface detector.<ref name="epa.gov">{{cite web|url=http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |title=UNITED STATES OF AMERICA, and STATE OF NORTH CAROLINA, Plaintiffs, v. PLANTATION PIPE LINE COMPANY, Civil Action No. 3:08-cv-500 |accessdate=June 14, 2012 |deadurl=yes |archiveurl=https://web.archive.org/web/20131028061446/http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |archivedate=October 28, 2013 |df= }}</ref>',
 '* 2000 On January 21, a [[Chevron Corporation|Chevron]] pipeline leaked from a welding flaw near [[Corinne, Utah]], spilling about 100 barrels of Diesel fuel. The product spread over 38 acres of salt flat and wetlands used by birds. About 75

In [1321]:
# splitted_text = text_rem_url.split('* ')
# splitted_text = splitted_text[1:]

In [1322]:
splitted_text[0]

'* 2000 On January 10, approximately 100 barrels of jet fuel were discharged from [[Plantation Pipeline]] in [[Newington, Virginia]], some of which entered into [[Accotink Creek]] and its adjoining shorelines. The failure resulted from a failed gasket on an interface detector.<ref name="epa.gov">{{cite web|url=http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |title=UNITED STATES OF AMERICA, and STATE OF NORTH CAROLINA, Plaintiffs, v. PLANTATION PIPE LINE COMPANY, Civil Action No. 3:08-cv-500 |accessdate=June 14, 2012 |deadurl=yes |archiveurl=https://web.archive.org/web/20131028061446/http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |archivedate=October 28, 2013 |df= }}</ref>'

In [1323]:
len(splitted_text)

351

In [1324]:
def extract_date(splitted_text):
    months = []
    new_months = []
    noon_index = []
    good_index = []
    lst = []
    for index,row in enumerate(splitted_text):
#         found_on = False
#         for i in row:
#         print(row)
        target = re.findall("^\* \d{4}.+[Oo]n [A-Z]\w+ \d+", row) # find row that has date
        if target != []:
            months.append(target)
            good_index.append(index) # get row that has date
#                 found_on = True # ignore row that doesn't declare date
#         if found_on == False :
#             lst.append(index)
    
    for v in good_index:
        lst.append(splitted_text[v])
        
    for row in months:
        target = re.sub(r'(?<=\d{4}).+[Oo]n', "", row[0].replace("* ",""))
        new_months.append(target)
    return new_months, lst

In [1325]:
dates, splitted_text = extract_date(splitted_text)

In [1326]:
len(splitted_text)

343

In [1327]:
len(dates)

343

In [1328]:
new_text = [i for i in splitted_text]
new_text

['* 2000 On January 10, approximately 100 barrels of jet fuel were discharged from [[Plantation Pipeline]] in [[Newington, Virginia]], some of which entered into [[Accotink Creek]] and its adjoining shorelines. The failure resulted from a failed gasket on an interface detector.<ref name="epa.gov">{{cite web|url=http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |title=UNITED STATES OF AMERICA, and STATE OF NORTH CAROLINA, Plaintiffs, v. PLANTATION PIPE LINE COMPANY, Civil Action No. 3:08-cv-500 |accessdate=June 14, 2012 |deadurl=yes |archiveurl=https://web.archive.org/web/20131028061446/http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |archivedate=October 28, 2013 |df= }}</ref>',
 '* 2000 On January 21, a [[Chevron Corporation|Chevron]] pipeline leaked from a welding flaw near [[Corinne, Utah]], spilling about 100 barrels of Diesel fuel. The product spread over 38 acres of salt flat and wetlands used by birds. About 75

In [1329]:
len(new_text)

343

## Extract people injured 

In [1330]:
df = pd.DataFrame(data=dates, columns=['Dates'])

In [1331]:
df.insert(1, 'Text', value=new_text)

In [1332]:
df.Text = df.Text.apply(lambda x: x.replace("* ", ""))

In [1333]:
# df['Text'] = df['Text'].apply(lambda x: re.sub(r'\[\[.+\]\]', "", str(x)))
df['Number of injuries'] = df['Text'].apply(lambda x: re.sub(r'\[\[.+\]\]', "", str(x)))
df['Number of injuries'] = df['Number of injuries'].apply(lambda x: re.sub(r'<ref>.*</ref>', "", str(x)))
df['Number of injuries'] = df['Number of injuries'].apply(lambda x: re.findall(r'[Nn][Oo]\s.*injuries|\..+injured', str(x)))
df['Number of injuries'] = df['Number of injuries'].apply(lambda x: str(x).replace("[","").replace("]","")
                                                                        .replace("\'",""))

In [1334]:
df.head(20)

Dates                                               Text  \
0    2000 January 10  2000 On January 10, approximately 100 barrels ...   
1    2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2    2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3    2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4    2000 February 5  2000 On February 5, a pipeline failed and spil...   
5       2000 March 9  2000 On March 9, an Explorer petroleum product...   
6       2000 April 7  2000 On April 7, a pipeline released fuel oil ...   
7        2000 May 19  2000 On May 19, a [[Colonial Pipeline]] Co. li...   
8        2000 June 7  2000 On June 7, a stopple fitting weld failed ...   
9        2000 July 5  2000 On July 5, two boats hit a [[Southern Nat...   
10      2000 July 26  2000 On August 19, a 30-inch diameter [[El Pas...   
11    2000 August 20  2000 On August 20, a gas pipeline exploded and...   
12    2000 August 24  2000 On August 24, a 6-inch pipeline operated ...   
13  2000 September 7  2000 On September 7, a bulldozer ruptured a 12...   
14   2000 November 3  2000 On November 3, a front end loader punctur...   
15   2001 January 17  2001 On January 17 and 18, a series of gas exp...   
16      2001 April 1  2001 On April 1, a Dome Pipeline in North Dako...   
17     2001 April 14  2001 On April 14, a 6-inch petroleum productio...   
18        2001 May 1  2001 On May 1, a MAPCO 10-inch propane pipelin...   
19       2001 May 24  2001 On May 24 bulldozer being used in [[Taylo...   

   Number of injuries  
0                      
1                      
2                      
3                      
4                      
5                      
6         No injuries  
7                      
8                      
9                      
10                     
11        no injuries  
12                     
13                     
14                     
15                     
16                     
17                     
18                     
19        no injuries

In [1335]:
df.Text[10]

'2000 On August 19, a 30-inch diameter [[El Paso Natural Gas]] pipeline rupture and fire near [[Carlsbad, New Mexico]] killed 12 members of an extended family camping over {{convert|600|ft|m}} from the rupture point. The force of the escaping gas created a {{convert|51|ft|m|adj=on}}-wide crater about {{convert|113|ft|m}} along the pipe. A {{convert|49|ft|m|adj=on}} section of the pipe was ejected from the crater, in three pieces measuring approximately {{convert|3|ft|m}}, {{convert|20|ft|m}}, and {{convert|26|ft|m}} in length. The largest piece of pipe was found about {{convert|287|ft|m}} northwest of the crater. The cause of the failure was determined to be severe internal corrosion of that pipeline. On July 26, 2007, a USDOJ Consent Decree was later entered into by the pipeline owner to perform pipeline system upgrades to allow better internal pipeline inspections.<ref>{{cite web|url=http://www.ntsb.gov/investigations/AccidentReports/Pages/PAR0301.aspx |title=Pipeline Accident Report

In [1336]:
df.Text[11]

'2000 On August 20, a gas pipeline exploded and burned in [[Concord, North Carolina]]. A nearby shopping mall was evacuated, but, there were no injuries.<ref>{{cite web|url=https://news.google.com/newspapers?nid=1314&dat=20000821&id=S9AzAAAAIBAJ&sjid=QvIDAAAAIBAJ&pg=6915,22149|title=The Spokesman-Review – Google News Archive Search|publisher=|accessdate=January 15, 2015}}</ref>'

In [1337]:
lst=Counter({'One':1, 'two':2, 'eight':8})
lst.keys()


dict_keys(['One', 'two', 'eight'])

In [1338]:
def get_people_injured(df, column_name):
    lst=Counter({'One':1, 'two':2, 'Eight':8, 'Four': 4})
    for index, i in enumerate(df[column_name]):
        if len(i) > 1:
            for j,v in enumerate(i.split(' ')):
                if v == 'no' or v == 'No':
                    df.loc[index,column_name] = 0
                    continue
                for number, value in lst.items():
                    if v == number:
                        df.loc[index,column_name] = value
    return df
df = get_people_injured(df, 'Number of injuries')

In [1339]:
# df['Number of injuries'] = df['Number of injuries'].apply(lambda x: re.findall(r'\d+',str(x)))
# df['Number of injuries'] = df['Number of injuries'].apply(lambda x: str(x).replace("[","").replace("]","").replace("'",""))
# df = df.replace("", 'UNKNOWN')

In [1340]:
df['Number of injuries'][df['Number of injuries'] != 0][df['Number of injuries'] != ""]

23                                                     1
76                                                     8
96                                                     4
108                                                    2
125                                                    1
140                                                    1
149    . The explosion ejected 106 feet of buried pip...
152                                                    2
169                                                    2
172                                                    1
181                                                    8
202                            . Two people were injured
204                           . Two workers were injured
258    . Nearly 1,000 residents were evacuated and at...
270                                        . The injured
293    . The resulting explosion killed 1 person and ...
311    . 4 employees were killed, and, one other injured
319              . 2 employees 

In [1341]:
df['Number of injuries'][102]

0

In [1342]:
df.head(50)

Dates                                               Text  \
0     2000 January 10  2000 On January 10, approximately 100 barrels ...   
1     2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2     2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3     2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4     2000 February 5  2000 On February 5, a pipeline failed and spil...   
5        2000 March 9  2000 On March 9, an Explorer petroleum product...   
6        2000 April 7  2000 On April 7, a pipeline released fuel oil ...   
7         2000 May 19  2000 On May 19, a [[Colonial Pipeline]] Co. li...   
8         2000 June 7  2000 On June 7, a stopple fitting weld failed ...   
9         2000 July 5  2000 On July 5, two boats hit a [[Southern Nat...   
10       2000 July 26  2000 On August 19, a 30-inch diameter [[El Pas...   
11     2000 August 20  2000 On August 20, a gas pipeline exploded and...   
12     2000 August 24  2000 On August 24, a 6-inch pipeline operated ...   
13   2000 September 7  2000 On September 7, a bulldozer ruptured a 12...   
14    2000 November 3  2000 On November 3, a front end loader punctur...   
15    2001 January 17  2001 On January 17 and 18, a series of gas exp...   
16       2001 April 1  2001 On April 1, a Dome Pipeline in North Dako...   
17      2001 April 14  2001 On April 14, a 6-inch petroleum productio...   
18         2001 May 1  2001 On May 1, a MAPCO 10-inch propane pipelin...   
19        2001 May 24  2001 On May 24 bulldozer being used in [[Taylo...   
20       2001 June 13  2001 On June 13, in [[Pensacola, Florida]], at...   
21       2001 July 24  2001 On July 24, a pipeline ruptured and sprea...   
22     2001 August 11  2001 On August 11, at approximately 5:05&nbsp;...   
23     2001 August 12  2001 On August 12, a bulldozer hit a 14-inch L...   
24     2001 August 17  2001 On August 17, an Oklahoma crude oil pipel...   
25   2001 September 3  2001 On September 3, at approximately 1&nbsp;p...   
26     2001 October 4  2001 On October 4, a drunken man used a rifle ...   
27    2001 October 15  2001 On October 15, a 6-inch [[ConocoPhillips]...   
28   2001 December 14  2001 On December 14, an anhydrous ammonia spil...   
29    2002 February 8  2002 On February 8, a trenching machine with a...   
30       2002 March 6  2002 On March 6, an explosion and a massive fi...   
31      2002 March 13  2002 On March 13, a Buckeye Partners pipeline ...   
32      2002 March 15  2002 On March 15, a failure occurred on a 36-i...   
33       2002 April 6  2002 On April 6, a BP-Amoco pipeline ruptured ...   
34        2002 May 22  2002 On May 22, an 8-inch petroleum products p...   
35       2002 June 20  2002 On June 20, PHMSA ordered [[Columbia Gas ...   
36        2002 July 4  2002 On July 4, there was a rupture of an [[En...   
37       2002 July 24  2002 On July 24 a gas explosion leveled a [[Ho...   
38      2002 August 5  2002 On August 5, a natural gas pipeline explo...   
39  2002 September 20  2002 On September 20, at around 22:10 a gasoli...   
40    2002 November 2  2002 On November 2, a [[Chevron Corporation|Ch...   
41   2002 December 10  2002 On December 10, a farmer plowing a field ...   
42    2003 January 24  2003 On January 24, an [[Enbridge]] crude oil ...   
43    2003 February 2  2003 On February 2, a natural gas pipeline rup...   
44   2003 February 20  2003 On February 20, a 24-inch gas transmissio...   
45   2003 February 27  2003 On February 27, dropping temperatures cau...   
46      2003 March 13  2003 On March 13, a seam failed on an 8-inch D...   
47      2003 March 23  2003 On March 23, a 24-inch [[El Paso Natural ...   
48       2003 April 1  2003 On April 1, a 12-inch [[ConocoPhillips]] ...   
49         2003 May 1  2003 On May 1, a 26-inch [[Williams Companies]...   

   Number of injuries  
0                      
1                      
2                      
3                      
4                  

## Extract deaths

In [1343]:
df['Deaths'] = df['Text'].apply(lambda x: re.sub(r'\[\[.+\]\]', "", str(x)))
df['Deaths'] = df['Deaths'].apply(lambda x: re.findall(r'[\.,].*kill[ed|ing][\s\w]+|[\.,].+[Dd]eaths.+[\.,]', str(x)))
df['Deaths'] = df['Deaths'].apply(lambda x: re.sub(r'url=.+',"", str(x)))

df['Deaths'] = df['Deaths'].apply(lambda x: str(x).replace("[","").replace("]","").replace("\'",""))

# df['Deaths'] = df['Deaths'].apply(lambda x: x.split(','))


In [1344]:
df.Deaths[119]

''

In [1345]:
df.head(20)

Dates                                               Text  \
0    2000 January 10  2000 On January 10, approximately 100 barrels ...   
1    2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2    2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3    2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4    2000 February 5  2000 On February 5, a pipeline failed and spil...   
5       2000 March 9  2000 On March 9, an Explorer petroleum product...   
6       2000 April 7  2000 On April 7, a pipeline released fuel oil ...   
7        2000 May 19  2000 On May 19, a [[Colonial Pipeline]] Co. li...   
8        2000 June 7  2000 On June 7, a stopple fitting weld failed ...   
9        2000 July 5  2000 On July 5, two boats hit a [[Southern Nat...   
10      2000 July 26  2000 On August 19, a 30-inch diameter [[El Pas...   
11    2000 August 20  2000 On August 20, a gas pipeline exploded and...   
12    2000 August 24  2000 On August 24, a 6-inch pipeline operated ...   
13  2000 September 7  2000 On September 7, a bulldozer ruptured a 12...   
14   2000 November 3  2000 On November 3, a front end loader punctur...   
15   2001 January 17  2001 On January 17 and 18, a series of gas exp...   
16      2001 April 1  2001 On April 1, a Dome Pipeline in North Dako...   
17     2001 April 14  2001 On April 14, a 6-inch petroleum productio...   
18        2001 May 1  2001 On May 1, a MAPCO 10-inch propane pipelin...   
19       2001 May 24  2001 On May 24 bulldozer being used in [[Taylo...   

   Number of injuries                                             Deaths  
0                                                                         
1                                                                         
2                                                                         
3                                                                         
4                                                                         
5                                                                         
6                   0                                                     
7                                                                         
8                                                                         
9                                                                         
10                     , a 30-inch diameter  killed 12 members of an ...  
11                  0                                                     
12                                                                        
13                                                                        
14                                                                        
15                     , a series of gas explosion hit downtown , res...  
16                                                                        
17                                                                        
18                                                                        
19                  0

In [1346]:
# def get_death(df, column_name):
#     for index, i in enumerate(df[column_name]):
#         if len(i) > 1:
#             for j,v in enumerate(i.split(' ')):
# #                 for x,v in enumerate(v.split(' ')):
#                 if v == 'killed' or v == 'killing' or v == 'deaths' or v == 'Deaths':
#                     df.loc[index,column_name] = v
#                     continue

In [1347]:
# deaths = get_death(df, 'Deaths')

In [1348]:
# for j,i in enumerate(['one','two','three','four','five','six','seven','eight','nine','ten']):
#     df['Deaths'] = df['Deaths'].apply(lambda x: str(x).replace(i, str(j+1)))

In [1349]:
df

Dates                                               Text  \
0      2000 January 10  2000 On January 10, approximately 100 barrels ...   
1      2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2      2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3      2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4      2000 February 5  2000 On February 5, a pipeline failed and spil...   
5         2000 March 9  2000 On March 9, an Explorer petroleum product...   
6         2000 April 7  2000 On April 7, a pipeline released fuel oil ...   
7          2000 May 19  2000 On May 19, a [[Colonial Pipeline]] Co. li...   
8          2000 June 7  2000 On June 7, a stopple fitting weld failed ...   
9          2000 July 5  2000 On July 5, two boats hit a [[Southern Nat...   
10        2000 July 26  2000 On August 19, a 30-inch diameter [[El Pas...   
11      2000 August 20  2000 On August 20, a gas pipeline exploded and...   
12      2000 August 24  2000 On August 24, a 6-inch pipeline operated ...   
13    2000 September 7  2000 On September 7, a bulldozer ruptured a 12...   
14     2000 November 3  2000 On November 3, a front end loader punctur...   
15     2001 January 17  2001 On January 17 and 18, a series of gas exp...   
16        2001 April 1  2001 On April 1, a Dome Pipeline in North Dako...   
17       2001 April 14  2001 On April 14, a 6-inch petroleum productio...   
18          2001 May 1  2001 On May 1, a MAPCO 10-inch propane pipelin...   
19         2001 May 24  2001 On May 24 bulldozer being used in [[Taylo...   
20        2001 June 13  2001 On June 13, in [[Pensacola, Florida]], at...   
21        2001 July 24  2001 On July 24, a pipeline ruptured and sprea...   
22      2001 August 11  2001 On August 11, at approximately 5:05&nbsp;...   
23      2001 August 12  2001 On August 12, a bulldozer hit a 14-inch L...   
24      2001 August 17  2001 On August 17, an Oklahoma crude oil pipel...   
25    2001 September 3  2001 On September 3, at approximately 1&nbsp;p...   
26      2001 October 4  2001 On October 4, a drunken man used a rifle ...   
27     2001 October 15  2001 On October 15, a 6-inch [[ConocoPhillips]...   
28    2001 December 14  2001 On December 14, an anhydrous ammonia spil...   
29     2002 February 8  2002 On February 8, a trenching machine with a...   
..                 ...                                                ...   
313   2015 November 30  2015 On November 30, about 11,000 gallons of g...   
314    2015 December 8  2015 On December 8, a contractor drilled into ...   
315     2016 January 2  2016 On January 2, 3 people were injured, one ...   
316     2016 January 9  2016 On January 9, a 30-inch [[Atmos Energy]] ...   
317    2016 January 11  2016 On January 11, butane leaking from a pipe...   
318   2016 February 14  2016 On February 14, a 6-inch crude oil pipeli...   
319   2016 February 16  2016 On February 16, an explosion and fire occ...   
320   2016 February 24  2016 On February 24, a 10-inch propane pipelin...   
321      2016 March 11  2016 On March 11, about 30,000 gallons of gaso...   
322      2016 March 22  2016 On March 22, about 4,000 gallons of gasol...   
323       2016 April 2  2016 On April 2, the [[TransCanada Corporation...   
324      2016 April 12  2016 On April 12, a pipeline at a gas plant in...   
325      2016 April 17  2016 On April 17, a 10 petroleum products pipe...   
326      2016 April 29  2016 On April 29, a 30-inch Texas Eastern/Spec...   
327        2016 May 20  2016 On May 20, a [[Shell Oil Company]] pipeli...   
328       2016 June 23  2016 On June 23, a Crimson Pipeline crude oil ...   
329        2016 July 6  2016 On July 6, a [[Plantation Pipeline]] line...   
330     2016 August 12  2016 On August 12, contractors were working on...   
331   2016 September 5  2016 On September 5, a pipeline in [[Bay Long,...   
332   2016 September 9  2016 On September 9, a [[Colonial Pipeline]] m...   
333  2016 Sep

In [1350]:
df.Deaths[119]

''

In [1351]:
# for j,i in enumerate(['one','two','three','four','five','six','seven','eight','nine','ten','a']):
df['Deaths'] = df['Deaths'].apply(lambda x: re.findall(r' \d+ ' ,str(x)))
df['Deaths'] = df['Deaths'].apply(lambda x: str(x).replace("[","").replace("]","").replace("'",""))
df = df.replace("", 'UNKNOWN')

In [1352]:
df.head(50)

Dates                                               Text  \
0     2000 January 10  2000 On January 10, approximately 100 barrels ...   
1     2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2     2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3     2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4     2000 February 5  2000 On February 5, a pipeline failed and spil...   
5        2000 March 9  2000 On March 9, an Explorer petroleum product...   
6        2000 April 7  2000 On April 7, a pipeline released fuel oil ...   
7         2000 May 19  2000 On May 19, a [[Colonial Pipeline]] Co. li...   
8         2000 June 7  2000 On June 7, a stopple fitting weld failed ...   
9         2000 July 5  2000 On July 5, two boats hit a [[Southern Nat...   
10       2000 July 26  2000 On August 19, a 30-inch diameter [[El Pas...   
11     2000 August 20  2000 On August 20, a gas pipeline exploded and...   
12     2000 August 24  2000 On August 24, a 6-inch pipeline operated ...   
13   2000 September 7  2000 On September 7, a bulldozer ruptured a 12...   
14    2000 November 3  2000 On November 3, a front end loader punctur...   
15    2001 January 17  2001 On January 17 and 18, a series of gas exp...   
16       2001 April 1  2001 On April 1, a Dome Pipeline in North Dako...   
17      2001 April 14  2001 On April 14, a 6-inch petroleum productio...   
18         2001 May 1  2001 On May 1, a MAPCO 10-inch propane pipelin...   
19        2001 May 24  2001 On May 24 bulldozer being used in [[Taylo...   
20       2001 June 13  2001 On June 13, in [[Pensacola, Florida]], at...   
21       2001 July 24  2001 On July 24, a pipeline ruptured and sprea...   
22     2001 August 11  2001 On August 11, at approximately 5:05&nbsp;...   
23     2001 August 12  2001 On August 12, a bulldozer hit a 14-inch L...   
24     2001 August 17  2001 On August 17, an Oklahoma crude oil pipel...   
25   2001 September 3  2001 On September 3, at approximately 1&nbsp;p...   
26     2001 October 4  2001 On October 4, a drunken man used a rifle ...   
27    2001 October 15  2001 On October 15, a 6-inch [[ConocoPhillips]...   
28   2001 December 14  2001 On December 14, an anhydrous ammonia spil...   
29    2002 February 8  2002 On February 8, a trenching machine with a...   
30       2002 March 6  2002 On March 6, an explosion and a massive fi...   
31      2002 March 13  2002 On March 13, a Buckeye Partners pipeline ...   
32      2002 March 15  2002 On March 15, a failure occurred on a 36-i...   
33       2002 April 6  2002 On April 6, a BP-Amoco pipeline ruptured ...   
34        2002 May 22  2002 On May 22, an 8-inch petroleum products p...   
35       2002 June 20  2002 On June 20, PHMSA ordered [[Columbia Gas ...   
36        2002 July 4  2002 On July 4, there was a rupture of an [[En...   
37       2002 July 24  2002 On July 24 a gas explosion leveled a [[Ho...   
38      2002 August 5  2002 On August 5, a natural gas pipeline explo...   
39  2002 September 20  2002 On September 20, at around 22:10 a gasoli...   
40    2002 November 2  2002 On November 2, a [[Chevron Corporation|Ch...   
41   2002 December 10  2002 On December 10, a farmer plowing a field ...   
42    2003 January 24  2003 On January 24, an [[Enbridge]] crude oil ...   
43    2003 February 2  2003 On February 2, a natural gas pipeline rup...   
44   2003 February 20  2003 On February 20, a 24-inch gas transmissio...   
45   2003 February 27  2003 On February 27, dropping temperatures cau...   
46      2003 March 13  2003 On March 13, a seam failed on an 8-inch D...   
47      2003 March 23  2003 On March 23, a 24-inch [[El Paso Natural ...   
48       2003 April 1  2003 On April 1, a 12-inch [[ConocoPhillips]] ...   
49         2003 May 1  2003 On May 1, a 26-inch [[Williams Companies]...   

   Number of injuries    Deaths  
0             UNKNOWN   UNKNOWN  
1             UNKNOWN   UNKNOWN  
2             UNKNOWN   UNKNOWN  
3  

## Extract state

In [1353]:
df['States'] = df['Text'].apply(lambda x: re.findall(r'\[\[[A-Z]\w+,\s[A-Z]\w+\]\]', str(x)))
# df['Deaths'] = df['Deaths'].apply(lambda x: re.findall(r'[\.,].+kill[ed|ing].+[\.,]', str(x)))

df['States'] = df['States'].apply(lambda x: str(x).replace("[","").replace("]","")
                                                                        .replace("\'",""))
df['States'] = df['States'].apply(lambda x: re.sub(r'\w+, ',"", str(x)))


In [1354]:
df.Text[0]

'2000 On January 10, approximately 100 barrels of jet fuel were discharged from [[Plantation Pipeline]] in [[Newington, Virginia]], some of which entered into [[Accotink Creek]] and its adjoining shorelines. The failure resulted from a failed gasket on an interface detector.<ref name="epa.gov">{{cite web|url=http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |title=UNITED STATES OF AMERICA, and STATE OF NORTH CAROLINA, Plaintiffs, v. PLANTATION PIPE LINE COMPANY, Civil Action No. 3:08-cv-500 |accessdate=June 14, 2012 |deadurl=yes |archiveurl=https://web.archive.org/web/20131028061446/http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf |archivedate=October 28, 2013 |df= }}</ref>'

In [1355]:
df.head()

Dates                                               Text  \
0  2000 January 10  2000 On January 10, approximately 100 barrels ...   
1  2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2  2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3  2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4  2000 February 5  2000 On February 5, a pipeline failed and spil...   

  Number of injuries   Deaths    States  
0            UNKNOWN  UNKNOWN  Virginia  
1            UNKNOWN  UNKNOWN      Utah  
2            UNKNOWN  UNKNOWN            
3            UNKNOWN  UNKNOWN  Kentucky  
4            UNKNOWN  UNKNOWN

## Extract links

In [1356]:
df.insert(5, 'Source', value=new_text)
df['Source'] = df['Source'].apply(lambda x: re.findall(r'(?<=url=).+title', x))
df['Source'] = df['Source'].apply(lambda x: str(x).replace('title',"").replace("[","").replace("]","").replace("|",'')
                                                                        .replace("\'","").replace(' ',""))
df

Dates                                               Text  \
0      2000 January 10  2000 On January 10, approximately 100 barrels ...   
1      2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2      2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3      2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4      2000 February 5  2000 On February 5, a pipeline failed and spil...   
5         2000 March 9  2000 On March 9, an Explorer petroleum product...   
6         2000 April 7  2000 On April 7, a pipeline released fuel oil ...   
7          2000 May 19  2000 On May 19, a [[Colonial Pipeline]] Co. li...   
8          2000 June 7  2000 On June 7, a stopple fitting weld failed ...   
9          2000 July 5  2000 On July 5, two boats hit a [[Southern Nat...   
10        2000 July 26  2000 On August 19, a 30-inch diameter [[El Pas...   
11      2000 August 20  2000 On August 20, a gas pipeline exploded and...   
12      2000 August 24  2000 On August 24, a 6-inch pipeline operated ...   
13    2000 September 7  2000 On September 7, a bulldozer ruptured a 12...   
14     2000 November 3  2000 On November 3, a front end loader punctur...   
15     2001 January 17  2001 On January 17 and 18, a series of gas exp...   
16        2001 April 1  2001 On April 1, a Dome Pipeline in North Dako...   
17       2001 April 14  2001 On April 14, a 6-inch petroleum productio...   
18          2001 May 1  2001 On May 1, a MAPCO 10-inch propane pipelin...   
19         2001 May 24  2001 On May 24 bulldozer being used in [[Taylo...   
20        2001 June 13  2001 On June 13, in [[Pensacola, Florida]], at...   
21        2001 July 24  2001 On July 24, a pipeline ruptured and sprea...   
22      2001 August 11  2001 On August 11, at approximately 5:05&nbsp;...   
23      2001 August 12  2001 On August 12, a bulldozer hit a 14-inch L...   
24      2001 August 17  2001 On August 17, an Oklahoma crude oil pipel...   
25    2001 September 3  2001 On September 3, at approximately 1&nbsp;p...   
26      2001 October 4  2001 On October 4, a drunken man used a rifle ...   
27     2001 October 15  2001 On October 15, a 6-inch [[ConocoPhillips]...   
28    2001 December 14  2001 On December 14, an anhydrous ammonia spil...   
29     2002 February 8  2002 On February 8, a trenching machine with a...   
..                 ...                                                ...   
313   2015 November 30  2015 On November 30, about 11,000 gallons of g...   
314    2015 December 8  2015 On December 8, a contractor drilled into ...   
315     2016 January 2  2016 On January 2, 3 people were injured, one ...   
316     2016 January 9  2016 On January 9, a 30-inch [[Atmos Energy]] ...   
317    2016 January 11  2016 On January 11, butane leaking from a pipe...   
318   2016 February 14  2016 On February 14, a 6-inch crude oil pipeli...   
319   2016 February 16  2016 On February 16, an explosion and fire occ...   
320   2016 February 24  2016 On February 24, a 10-inch propane pipelin...   
321      2016 March 11  2016 On March 11, about 30,000 gallons of gaso...   
322      2016 March 22  2016 On March 22, about 4,000 gallons of gasol...   
323       2016 April 2  2016 On April 2, the [[TransCanada Corporation...   
324      2016 April 12  2016 On April 12, a pipeline at a gas plant in...   
325      2016 April 17  2016 On April 17, a 10 petroleum products pipe...   
326      2016 April 29  2016 On April 29, a 30-inch Texas Eastern/Spec...   
327        2016 May 20  2016 On May 20, a [[Shell Oil Company]] pipeli...   
328       2016 June 23  2016 On June 23, a Crimson Pipeline crude oil ...   
329        2016 July 6  2016 On July 6, a [[Plantation Pipeline]] line...   
330     2016 August 12  2016 On August 12, contractors were working on...   
331   2016 September 5  2016 On September 5, a pipeline in [[Bay Long,...   
332   2016 September 9  2016 On September 9, a [[Colonial Pipeline]] m...   
333  2016 Sep

In [1357]:
df.Source[0]

'http://www.epa.gov/compliance/resources/decrees/civil/cwa/plantationpipeline-cd.pdf'

## Extract units

In [1358]:
df['Spill Amount (Barrels)'] = df['Text'].apply(lambda x: str(x).replace(',',''))
df['Spill Amount (Barrels)'] = df['Spill Amount (Barrels)'].apply(lambda x: re.findall(r'\d+ barrels', str(x)))

df['Spill Amount (Barrels)'] = df['Spill Amount (Barrels)'].apply(lambda x: str(x).replace("[","").replace("]","").replace("|",'')
                                                                        .replace("\'",""))

In [1359]:
# df['Spill Amount (Barrels)'] = df['Spill Amount (Barrels)'].apply(lambda x: str(x).split(' '))

In [1360]:
df['Spill Amount (Barrels)'] = df['Spill Amount (Barrels)'].apply(lambda x: x.replace(' barrels', ""))

In [1361]:
df['Spill Amount (Barrels)'] = df['Spill Amount (Barrels)'].apply(lambda x: int(x) if re.findall(r'barrels',str(x)) == False else x)

In [1362]:
df

Dates                                               Text  \
0      2000 January 10  2000 On January 10, approximately 100 barrels ...   
1      2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2      2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3      2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4      2000 February 5  2000 On February 5, a pipeline failed and spil...   
5         2000 March 9  2000 On March 9, an Explorer petroleum product...   
6         2000 April 7  2000 On April 7, a pipeline released fuel oil ...   
7          2000 May 19  2000 On May 19, a [[Colonial Pipeline]] Co. li...   
8          2000 June 7  2000 On June 7, a stopple fitting weld failed ...   
9          2000 July 5  2000 On July 5, two boats hit a [[Southern Nat...   
10        2000 July 26  2000 On August 19, a 30-inch diameter [[El Pas...   
11      2000 August 20  2000 On August 20, a gas pipeline exploded and...   
12      2000 August 24  2000 On August 24, a 6-inch pipeline operated ...   
13    2000 September 7  2000 On September 7, a bulldozer ruptured a 12...   
14     2000 November 3  2000 On November 3, a front end loader punctur...   
15     2001 January 17  2001 On January 17 and 18, a series of gas exp...   
16        2001 April 1  2001 On April 1, a Dome Pipeline in North Dako...   
17       2001 April 14  2001 On April 14, a 6-inch petroleum productio...   
18          2001 May 1  2001 On May 1, a MAPCO 10-inch propane pipelin...   
19         2001 May 24  2001 On May 24 bulldozer being used in [[Taylo...   
20        2001 June 13  2001 On June 13, in [[Pensacola, Florida]], at...   
21        2001 July 24  2001 On July 24, a pipeline ruptured and sprea...   
22      2001 August 11  2001 On August 11, at approximately 5:05&nbsp;...   
23      2001 August 12  2001 On August 12, a bulldozer hit a 14-inch L...   
24      2001 August 17  2001 On August 17, an Oklahoma crude oil pipel...   
25    2001 September 3  2001 On September 3, at approximately 1&nbsp;p...   
26      2001 October 4  2001 On October 4, a drunken man used a rifle ...   
27     2001 October 15  2001 On October 15, a 6-inch [[ConocoPhillips]...   
28    2001 December 14  2001 On December 14, an anhydrous ammonia spil...   
29     2002 February 8  2002 On February 8, a trenching machine with a...   
..                 ...                                                ...   
313   2015 November 30  2015 On November 30, about 11,000 gallons of g...   
314    2015 December 8  2015 On December 8, a contractor drilled into ...   
315     2016 January 2  2016 On January 2, 3 people were injured, one ...   
316     2016 January 9  2016 On January 9, a 30-inch [[Atmos Energy]] ...   
317    2016 January 11  2016 On January 11, butane leaking from a pipe...   
318   2016 February 14  2016 On February 14, a 6-inch crude oil pipeli...   
319   2016 February 16  2016 On February 16, an explosion and fire occ...   
320   2016 February 24  2016 On February 24, a 10-inch propane pipelin...   
321      2016 March 11  2016 On March 11, about 30,000 gallons of gaso...   
322      2016 March 22  2016 On March 22, about 4,000 gallons of gasol...   
323       2016 April 2  2016 On April 2, the [[TransCanada Corporation...   
324      2016 April 12  2016 On April 12, a pipeline at a gas plant in...   
325      2016 April 17  2016 On April 17, a 10 petroleum products pipe...   
326      2016 April 29  2016 On April 29, a 30-inch Texas Eastern/Spec...   
327        2016 May 20  2016 On May 20, a [[Shell Oil Company]] pipeli...   
328       2016 June 23  2016 On June 23, a Crimson Pipeline crude oil ...   
329        2016 July 6  2016 On July 6, a [[Plantation Pipeline]] line...   
330     2016 August 12  2016 On August 12, contractors were working on...   
331   2016 September 5  2016 On September 5, a pipeline in [[Bay Long,...   
332   2016 September 9  2016 On September 9, a [[Colonial Pipeline]] m...   
333  2016 Sep

## Extract oil spill in Gallons

In [1363]:
df['Spill Amount (Gallons)'] = df['Text'].apply(lambda x: str(x).replace(',',''))
df['Spill Amount (Gallons)'] = df['Spill Amount (Gallons)'].apply(lambda x: re.findall(r'(?=\d+ gallons)\d+', str(x)))

df['Spill Amount (Gallons)'] = df['Spill Amount (Gallons)'].apply(lambda x: str(x).replace("[","").replace("]","").replace("|",'')
                                                                        .replace("\'","").replace(" ",""))

In [1364]:
df['Spill Amount (Gallons)'] = df['Spill Amount (Gallons)'].apply(lambda x: str(x).split(','))


In [1365]:
df['Spill Amount (Gallons)'] = df['Spill Amount (Gallons)'].apply(lambda x: min(x[0],x[1]) if len(x) >= 2 else x)
df['Spill Amount (Gallons)'] = df['Spill Amount (Gallons)'].apply(lambda x: str(x).replace("[","").replace("]","").replace("|",'')
                                                                        .replace("\'","").replace(" ",""))

In [1366]:
df['Spill Amount (Gallons)'] = df['Spill Amount (Gallons)'].apply(lambda x: float(x) if x!='' else x)



In [1367]:
df

Dates                                               Text  \
0      2000 January 10  2000 On January 10, approximately 100 barrels ...   
1      2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2      2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3      2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4      2000 February 5  2000 On February 5, a pipeline failed and spil...   
5         2000 March 9  2000 On March 9, an Explorer petroleum product...   
6         2000 April 7  2000 On April 7, a pipeline released fuel oil ...   
7          2000 May 19  2000 On May 19, a [[Colonial Pipeline]] Co. li...   
8          2000 June 7  2000 On June 7, a stopple fitting weld failed ...   
9          2000 July 5  2000 On July 5, two boats hit a [[Southern Nat...   
10        2000 July 26  2000 On August 19, a 30-inch diameter [[El Pas...   
11      2000 August 20  2000 On August 20, a gas pipeline exploded and...   
12      2000 August 24  2000 On August 24, a 6-inch pipeline operated ...   
13    2000 September 7  2000 On September 7, a bulldozer ruptured a 12...   
14     2000 November 3  2000 On November 3, a front end loader punctur...   
15     2001 January 17  2001 On January 17 and 18, a series of gas exp...   
16        2001 April 1  2001 On April 1, a Dome Pipeline in North Dako...   
17       2001 April 14  2001 On April 14, a 6-inch petroleum productio...   
18          2001 May 1  2001 On May 1, a MAPCO 10-inch propane pipelin...   
19         2001 May 24  2001 On May 24 bulldozer being used in [[Taylo...   
20        2001 June 13  2001 On June 13, in [[Pensacola, Florida]], at...   
21        2001 July 24  2001 On July 24, a pipeline ruptured and sprea...   
22      2001 August 11  2001 On August 11, at approximately 5:05&nbsp;...   
23      2001 August 12  2001 On August 12, a bulldozer hit a 14-inch L...   
24      2001 August 17  2001 On August 17, an Oklahoma crude oil pipel...   
25    2001 September 3  2001 On September 3, at approximately 1&nbsp;p...   
26      2001 October 4  2001 On October 4, a drunken man used a rifle ...   
27     2001 October 15  2001 On October 15, a 6-inch [[ConocoPhillips]...   
28    2001 December 14  2001 On December 14, an anhydrous ammonia spil...   
29     2002 February 8  2002 On February 8, a trenching machine with a...   
..                 ...                                                ...   
313   2015 November 30  2015 On November 30, about 11,000 gallons of g...   
314    2015 December 8  2015 On December 8, a contractor drilled into ...   
315     2016 January 2  2016 On January 2, 3 people were injured, one ...   
316     2016 January 9  2016 On January 9, a 30-inch [[Atmos Energy]] ...   
317    2016 January 11  2016 On January 11, butane leaking from a pipe...   
318   2016 February 14  2016 On February 14, a 6-inch crude oil pipeli...   
319   2016 February 16  2016 On February 16, an explosion and fire occ...   
320   2016 February 24  2016 On February 24, a 10-inch propane pipelin...   
321      2016 March 11  2016 On March 11, about 30,000 gallons of gaso...   
322      2016 March 22  2016 On March 22, about 4,000 gallons of gasol...   
323       2016 April 2  2016 On April 2, the [[TransCanada Corporation...   
324      2016 April 12  2016 On April 12, a pipeline at a gas plant in...   
325      2016 April 17  2016 On April 17, a 10 petroleum products pipe...   
326      2016 April 29  2016 On April 29, a 30-inch Texas Eastern/Spec...   
327        2016 May 20  2016 On May 20, a [[Shell Oil Company]] pipeli...   
328       2016 June 23  2016 On June 23, a Crimson Pipeline crude oil ...   
329        2016 July 6  2016 On July 6, a [[Plantation Pipeline]] line...   
330     2016 August 12  2016 On August 12, contractors were working on...   
331   2016 September 5  2016 On September 5, a pipeline in [[Bay Long,...   
332   2016 September 9  2016 On September 9, a [[Colonial Pipeline]] m...   
333  2016 Sep

In [1368]:
# df['Spill Amount (Barrels)'] = df['Spill Amount (Gallons)'].apply(lambda x: int(x) if re.findall(r'barrels',str(x)) == False else x)

In [1369]:
df['Text'][122]

'2007 On November 12, three teenaged boys drilled into an ammonia pipeline, in [[Tampa Bay, Florida]], causing a major ammonia leak. They later claimed they did it due to stories of money being hidden inside that pipeline. The leak took two days to be capped. One of the teens had serious chemical burns from the ammonia. Residents within a half miles from the leak were evacuated. PHMSA later noted the pipeline company failed to adequately plan for emergencies with the local Fire Agency, as required by CFR 195.402(c)(12).<ref>{{cite web|title=CPF No. 2-2008-6002 |url=http://primis.phmsa.dot.gov/comm/reports/enforce/documents/220086002/220086002_FinalOrder_04262010.pdf |accessdate=27 November 2016 |deadurl=no |archiveurl=https://web.archive.org/web/20170211114341/https://primis.phmsa.dot.gov/comm/reports/enforce/documents/220086002/220086002_FinalOrder_04262010.pdf |archivedate=February 11, 2017 }}</ref>'

## Extract cause of loss

In [1374]:
df['Cause of Loss'] = df['Text'].apply(lambda x: str(x).replace(',',''))
df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: re.findall(r'leak|ruptur|explod|fire|puncture|spill|material defect', str(x)))

df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: str(x).replace("[","").replace("]","").replace("|",'')
                                                                        .replace("\'","").replace(" ",""))

In [1375]:
# df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: str(x).split(','))


In [1376]:
df

Dates                                               Text  \
0      2000 January 10  2000 On January 10, approximately 100 barrels ...   
1      2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2      2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3      2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4      2000 February 5  2000 On February 5, a pipeline failed and spil...   
5         2000 March 9  2000 On March 9, an Explorer petroleum product...   
6         2000 April 7  2000 On April 7, a pipeline released fuel oil ...   
7          2000 May 19  2000 On May 19, a [[Colonial Pipeline]] Co. li...   
8          2000 June 7  2000 On June 7, a stopple fitting weld failed ...   
9          2000 July 5  2000 On July 5, two boats hit a [[Southern Nat...   
10        2000 July 26  2000 On August 19, a 30-inch diameter [[El Pas...   
11      2000 August 20  2000 On August 20, a gas pipeline exploded and...   
12      2000 August 24  2000 On August 24, a 6-inch pipeline operated ...   
13    2000 September 7  2000 On September 7, a bulldozer ruptured a 12...   
14     2000 November 3  2000 On November 3, a front end loader punctur...   
15     2001 January 17  2001 On January 17 and 18, a series of gas exp...   
16        2001 April 1  2001 On April 1, a Dome Pipeline in North Dako...   
17       2001 April 14  2001 On April 14, a 6-inch petroleum productio...   
18          2001 May 1  2001 On May 1, a MAPCO 10-inch propane pipelin...   
19         2001 May 24  2001 On May 24 bulldozer being used in [[Taylo...   
20        2001 June 13  2001 On June 13, in [[Pensacola, Florida]], at...   
21        2001 July 24  2001 On July 24, a pipeline ruptured and sprea...   
22      2001 August 11  2001 On August 11, at approximately 5:05&nbsp;...   
23      2001 August 12  2001 On August 12, a bulldozer hit a 14-inch L...   
24      2001 August 17  2001 On August 17, an Oklahoma crude oil pipel...   
25    2001 September 3  2001 On September 3, at approximately 1&nbsp;p...   
26      2001 October 4  2001 On October 4, a drunken man used a rifle ...   
27     2001 October 15  2001 On October 15, a 6-inch [[ConocoPhillips]...   
28    2001 December 14  2001 On December 14, an anhydrous ammonia spil...   
29     2002 February 8  2002 On February 8, a trenching machine with a...   
..                 ...                                                ...   
313   2015 November 30  2015 On November 30, about 11,000 gallons of g...   
314    2015 December 8  2015 On December 8, a contractor drilled into ...   
315     2016 January 2  2016 On January 2, 3 people were injured, one ...   
316     2016 January 9  2016 On January 9, a 30-inch [[Atmos Energy]] ...   
317    2016 January 11  2016 On January 11, butane leaking from a pipe...   
318   2016 February 14  2016 On February 14, a 6-inch crude oil pipeli...   
319   2016 February 16  2016 On February 16, an explosion and fire occ...   
320   2016 February 24  2016 On February 24, a 10-inch propane pipelin...   
321      2016 March 11  2016 On March 11, about 30,000 gallons of gaso...   
322      2016 March 22  2016 On March 22, about 4,000 gallons of gasol...   
323       2016 April 2  2016 On April 2, the [[TransCanada Corporation...   
324      2016 April 12  2016 On April 12, a pipeline at a gas plant in...   
325      2016 April 17  2016 On April 17, a 10 petroleum products pipe...   
326      2016 April 29  2016 On April 29, a 30-inch Texas Eastern/Spec...   
327        2016 May 20  2016 On May 20, a [[Shell Oil Company]] pipeli...   
328       2016 June 23  2016 On June 23, a Crimson Pipeline crude oil ...   
329        2016 July 6  2016 On July 6, a [[Plantation Pipeline]] line...   
330     2016 August 12  2016 On August 12, contractors were working on...   
331   2016 September 5  2016 On September 5, a pipeline in [[Bay Long,...   
332   2016 September 9  2016 On September 9, a [[Colonial Pipeline]] m...   
333  2016 Sep

In [1377]:
df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: 'Leaks and Spills' if re.findall(r'leak|spill',x) else x)
df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: 'Pipeline Rupture' if re.findall(r'ruptur|puncture',x) else x)
df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: 'Explosion' if re.findall(r'explod',x) else x)
df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: 'Fire/Blaze' if re.findall(r'fire',x) else x)
df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: 'Fire/Blaze' if re.findall(r'fire',x) else x)
df['Cause of Loss'] = df['Cause of Loss'].apply(lambda x: 'Maintenance' if re.findall(r'material defect',x) else x)





In [1378]:
df

Dates                                               Text  \
0      2000 January 10  2000 On January 10, approximately 100 barrels ...   
1      2000 January 21  2000 On January 21, a [[Chevron Corporation|Ch...   
2      2000 January 21  2000 On January 21, an Equilon Pipeline Co. cr...   
3      2000 January 27  2000 On January 27, in [[Winchester, Kentucky]...   
4      2000 February 5  2000 On February 5, a pipeline failed and spil...   
5         2000 March 9  2000 On March 9, an Explorer petroleum product...   
6         2000 April 7  2000 On April 7, a pipeline released fuel oil ...   
7          2000 May 19  2000 On May 19, a [[Colonial Pipeline]] Co. li...   
8          2000 June 7  2000 On June 7, a stopple fitting weld failed ...   
9          2000 July 5  2000 On July 5, two boats hit a [[Southern Nat...   
10        2000 July 26  2000 On August 19, a 30-inch diameter [[El Pas...   
11      2000 August 20  2000 On August 20, a gas pipeline exploded and...   
12      2000 August 24  2000 On August 24, a 6-inch pipeline operated ...   
13    2000 September 7  2000 On September 7, a bulldozer ruptured a 12...   
14     2000 November 3  2000 On November 3, a front end loader punctur...   
15     2001 January 17  2001 On January 17 and 18, a series of gas exp...   
16        2001 April 1  2001 On April 1, a Dome Pipeline in North Dako...   
17       2001 April 14  2001 On April 14, a 6-inch petroleum productio...   
18          2001 May 1  2001 On May 1, a MAPCO 10-inch propane pipelin...   
19         2001 May 24  2001 On May 24 bulldozer being used in [[Taylo...   
20        2001 June 13  2001 On June 13, in [[Pensacola, Florida]], at...   
21        2001 July 24  2001 On July 24, a pipeline ruptured and sprea...   
22      2001 August 11  2001 On August 11, at approximately 5:05&nbsp;...   
23      2001 August 12  2001 On August 12, a bulldozer hit a 14-inch L...   
24      2001 August 17  2001 On August 17, an Oklahoma crude oil pipel...   
25    2001 September 3  2001 On September 3, at approximately 1&nbsp;p...   
26      2001 October 4  2001 On October 4, a drunken man used a rifle ...   
27     2001 October 15  2001 On October 15, a 6-inch [[ConocoPhillips]...   
28    2001 December 14  2001 On December 14, an anhydrous ammonia spil...   
29     2002 February 8  2002 On February 8, a trenching machine with a...   
..                 ...                                                ...   
313   2015 November 30  2015 On November 30, about 11,000 gallons of g...   
314    2015 December 8  2015 On December 8, a contractor drilled into ...   
315     2016 January 2  2016 On January 2, 3 people were injured, one ...   
316     2016 January 9  2016 On January 9, a 30-inch [[Atmos Energy]] ...   
317    2016 January 11  2016 On January 11, butane leaking from a pipe...   
318   2016 February 14  2016 On February 14, a 6-inch crude oil pipeli...   
319   2016 February 16  2016 On February 16, an explosion and fire occ...   
320   2016 February 24  2016 On February 24, a 10-inch propane pipelin...   
321      2016 March 11  2016 On March 11, about 30,000 gallons of gaso...   
322      2016 March 22  2016 On March 22, about 4,000 gallons of gasol...   
323       2016 April 2  2016 On April 2, the [[TransCanada Corporation...   
324      2016 April 12  2016 On April 12, a pipeline at a gas plant in...   
325      2016 April 17  2016 On April 17, a 10 petroleum products pipe...   
326      2016 April 29  2016 On April 29, a 30-inch Texas Eastern/Spec...   
327        2016 May 20  2016 On May 20, a [[Shell Oil Company]] pipeli...   
328       2016 June 23  2016 On June 23, a Crimson Pipeline crude oil ...   
329        2016 July 6  2016 On July 6, a [[Plantation Pipeline]] line...   
330     2016 August 12  2016 On August 12, contractors were working on...   
331   2016 September 5  2016 On September 5, a pipeline in [[Bay Long,...   
332   2016 September 9  2016 On September 9, a [[Colonial Pipeline]] m...   
333  2016 Sep

# Put all metrics into dataframe

In [1379]:
# df['Cause of Loss'] = 'Leaks and Spills'
df['Country'] = 'USA'
df['Name of Owner'] = 'UNKNOWN'
df['Name of Installation'] = 'UNKNOWN'
df = df.rename(index=str, columns={'Dates': 'Incident Date', 'States': 'State'})

In [1380]:
df = pd.DataFrame(data=df, columns=['Incident Date', 'Country', 'State', 'Cause of Loss', 'Name of Installation', 'Name of Owner',
                                    'Text', 'Number of injuries', 'Deaths', 'Spill Amount (Barrels)', 'Spill Amount (Gallons)','Source'])

In [1381]:
# df['Incident Date'] =  pd.to_datetime(df['Incident Date'], format='%Y %b %d')
# # df['Incident Date'].dt.datetime

In [1382]:
df['Deaths'][df.Deaths != 'UNKNOWN']

10           12 
15       2 ,  2 
163           5 
254           3 
293     1 ,  12 
311           4 
324           2 
Name: Deaths, dtype: object

In [1383]:
df.tail(15)

Incident Date Country     State     Cause of Loss  \
328       2016 June 23     USA            Leaks and Spills   
329        2016 July 6     USA            Leaks and Spills   
330     2016 August 12     USA     Texas        Fire/Blaze   
331   2016 September 5     USA            Leaks and Spills   
332   2016 September 9     USA            Leaks and Spills   
333  2016 September 10     USA     Texas  Leaks and Spills   
334    2016 October 11     USA  Illinois  Leaks and Spills   
335    2016 October 17     USA  Nebraska  Leaks and Spills   
336    2016 October 19     USA    Oregon        Fire/Blaze   
337    2016 October 21     USA            Leaks and Spills   
338    2016 October 23     USA  Oklahoma  Leaks and Spills   
339    2016 October 31     USA                   Explosion   
340   2016 November 29     USA                   Explosion   
341    2016 December 2     USA            Leaks and Spills   
342    2016 December 5     USA            Leaks and Spills   

    Name of Installation Name of Owner  \
328              UNKNOWN       UNKNOWN   
329              UNKNOWN       UNKNOWN   
330              UNKNOWN       UNKNOWN   
331              UNKNOWN       UNKNOWN   
332              UNKNOWN       UNKNOWN   
333              UNKNOWN       UNKNOWN   
334              UNKNOWN       UNKNOWN   
335              UNKNOWN       UNKNOWN   
336              UNKNOWN       UNKNOWN   
337              UNKNOWN       UNKNOWN   
338              UNKNOWN       UNKNOWN   
339              UNKNOWN       UNKNOWN   
340              UNKNOWN       UNKNOWN   
341              UNKNOWN       UNKNOWN   
342              UNKNOWN       UNKNOWN   

                                                  Text  \
328  2016 On June 23, a Crimson Pipeline crude oil ...   
329  2016 On July 6, a [[Plantation Pipeline]] line...   
330  2016 On August 12, contractors were working on...   
331  2016 On September 5, a pipeline in [[Bay Long,...   
332  2016 On September 9, a [[Colonial Pipeline]] m...   
333  2016 On September 10, a [[Sunoco]] pipeline ru...   
334  2016 On October 11, two Nicor Gas workers were...   
335  2016 On October 17, an 8-inch ammonia pipeline...   
336  2016 On October 19, a contractor in [[Portland...   
337  2016 On October 21, an 8-inch Sunoco pipeline ...   
338  2016 On October 23, a pipeline ruptured on the...   
339  2016 On October 31, a [[Colonial Pipeline]] ma...   
340  2016 On November 29, an Enterprise Products pi...   
341  2016 On December 2, equipment failure in a [[D...   
342  2016 On December 5, a 6-inch Belle Fourche pip...   

                                    Number of injuries   Deaths  \
328                                            UNKNOWN  UNKNOWN   
329                                            UNKNOWN  UNKNOWN   
330  . The contractors were knocked off the platfor...  UNKNOWN   
331                                            UNKNOWN  UNKNOWN   
332                                            UNKNOWN  UNKNOWN   
333                                            UNKNOWN  UNKNOWN   
334                                            UNKNOWN  UNKNOWN   
335                                            UNKNOWN  UNKNOWN   
336                                            UNKNOWN  UNKNOWN   
337                                            UNKNOWN  UNKNOWN   
338                                            UNKNOWN  UNKNOWN   
339                                            UNKNOWN  UNKNOWN   
340                                                  0  UNKNOWN   
341                                            UNKNOWN  UNKNOWN   
342                                            UNKNOWN  UNKNOWN   

    Spill Amount (Barrels) Spill Amount (Gallons)  \
328                                         25200   
329                                                 
330                                                 
331                                          5300   
332                                        250000   
333                    

In [1384]:
writer = pd.ExcelWriter('oil_spill_list_2000_2017.xlsx')
df.to_excel(writer,'oil_spill_list2000-2017')
writer.save()

/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'http://ioscproceedings.org/doi/pdf/10.7901/2169-3358-2003-1-109=UseofInSituBurningataDieselSpillinWetlandsandSaltFlats,NorthernUtah,U.S.A:RemediationOperationsand1.5YearsofPost-BurnMonitoringpublisher=date=accessdate=April12,2014%7d%7d%3c/ref%3e%3cref%3e%7b%7bciteweburl=http://projects.propublica.org/pipelines/incidents/270220' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/Tai/anaconda3/envs/nlp/lib/python3.6/site-packages/xlsxwriter/worksheet.py:831: UserWarning: Ignoring URL 'http://projects.propublica.org/pipelines/incidents/270221=Jan.21,2000–ProPublicapublisher=accessdate=January15,2015deadurl=noarchiveurl=https://web.archive.org/web/20141020202700/http://projects.propublica.org/pipelines/incidents/270221archivedate=October20,2014%7d%7d%3c/ref%3e%3cref%3e%7b%7bciteweburl=https://news.google.com/newspa